In [1]:
import sys
from ezr import *

In [2]:
import random
import matplotlib.pyplot as plt
import numpy as np
import os

config_folder = 'data/optimize/config'
datasets = [os.path.join(config_folder, file) for file in os.listdir(config_folder) if file.endswith('.csv')]

In [3]:
import random

def activeLearning(self: DATA, score=lambda B, R: B - R, generate=None, faster=True):
    def ranked(rows):
        return self.clone(rows).distances().rows

    def todos(todo):
        if faster:  # Apply sorting heuristics to a small buffer at the start of "todo"
            n = the.buffer // 2
            return todo[:n] + todo[2 * n: 3 * n], todo[3 * n:] + todo[n:2 * n]
        else:  # Apply sorting heuristics to all of "todo"
            return todo, []

    def guess(todo: rows, done: rows) -> rows:
        cut = int(.5 + len(done) ** the.cut)
        best = self.clone(done[:cut])
        rest = self.clone(done[cut:])
        a, b = todos(todo)
        if generate:
            return self.neighbors(generate(best, rest), a) + b
        else:
            key = lambda r: score(best.loglike(r, len(done), 2), rest.loglike(r, len(done), 2))
            return sorted(a, key=key, reverse=True) + b

    def guess_all(todo: rows, done: rows) -> rows:
        cut = int(.5 + len(done) ** the.cut)
        best = self.clone(done[:cut])
        rest = self.clone(done[cut:])
        key = lambda r: best.loglike(r, len(done), 2)
        return sorted(todo, key=key, reverse=True)

    def loop(todo: rows, done: rows) -> rows:
        iteration_results = []
        while len(todo) > 2 and len(done) < the.Last:
            ranking_result = model_ranking(todo, done)
            print(f'{the.iter}. {ranking_result}')
            iteration_results.append(ranking_result)
            top, *todo = guess(todo, done)
            done += [top]
            done = ranked(done)
            the.iter += 1
        iteration_results.append(model_ranking(todo, done))
        return done, iteration_results

    todo, done = self.rows[the.label:], ranked(self.rows[:the.label])

    if the.branch == True:
        todo, done = self.branch(used=[])
        if the.Last == 0: return done

    
    def model_ranking(todo, done):
        # Determine the cutoff for the best subset
        cut = int(0.5 + len(done) ** the.cut)
        best = self.clone(done[:cut])  # Select the best subset
        rest = self.clone(done[cut:])  # The remaining rows
        
        # Generate 1000 random pairs
        random_pairs = [(random.choice(d.rows), random.choice(d.rows)) for _ in range(100)]
        
        correct_guesses = 0
        
        for row1, row2 in random_pairs:
            # Use loglike to guess which row has a higher y value
            guess = row1 if best.loglike(row1, len(done), 2) > best.loglike(row2, len(done), 2) else row2
            
            # Find true y values using d.distance(row)
            true_higher = row1 if d.distance(row1) < d.distance(row2) else row2
            
            # Check if the guess is correct
            if guess == true_higher:
                correct_guesses += 1
        
        return correct_guesses


    the.iter = 0
    return loop(todo, done)

# Exploit with Random Initialization

In [11]:
# Load dataset and initialize
dataset = 'data/optimize/misc/auto93.csv'
d = DATA().adds(csv(dataset))

print(dataset)
print(len(d.rows))

def exploit(B, R):
    return B

def explore(B, R):
    return (B + R) / (abs(B - R) + 10 ** -30)

the.Last = 40
first_iterations = []
last_iterations = []

the.branch = False
random.seed(10)

# Run trials
for trial in range(100):
    print(f'\nTrial {trial}')
    random.shuffle(d.rows)
    result, iteration_results = activeLearning(d, lambda B, R: exploit(B, R))

    if iteration_results:  # Store the first and last iteration results
        first_iterations.append(iteration_results[0])
        last_iterations.append(iteration_results[-1])

# Calculate and print the average values
average_first = sum(first_iterations) / len(first_iterations)
average_last = sum(last_iterations) / len(last_iterations)

print(f'\nAverage value of the first iteration across 100 trials: {average_first}')
print(f'Average value of the last iteration across 100 trials: {average_last}')

data/optimize/misc/auto93.csv
398

Trial 0
0. 75
1. 72
2. 76
3. 77
4. 75
5. 69
6. 77
7. 73
8. 61
9. 76
10. 78
11. 72
12. 77
13. 74
14. 76
15. 77
16. 61
17. 73
18. 70
19. 82
20. 76
21. 73
22. 70
23. 75
24. 68
25. 72
26. 72
27. 80
28. 75
29. 73
30. 75
31. 78
32. 71
33. 78
34. 75
35. 75

Trial 1
0. 81
1. 76
2. 81
3. 73
4. 73
5. 74
6. 83
7. 77
8. 75
9. 79
10. 75
11. 71
12. 83
13. 78
14. 74
15. 77
16. 75
17. 82
18. 78
19. 78
20. 80
21. 75
22. 80
23. 84
24. 79
25. 77
26. 81
27. 79
28. 83
29. 86
30. 79
31. 82
32. 86
33. 75
34. 78
35. 70

Trial 2
0. 76
1. 74
2. 76
3. 78
4. 71
5. 78
6. 64
7. 73
8. 65
9. 71
10. 76
11. 72
12. 65
13. 75
14. 72
15. 69
16. 73
17. 79
18. 71
19. 78
20. 73
21. 73
22. 64
23. 75
24. 79
25. 68
26. 68
27. 73
28. 83
29. 81
30. 72
31. 82
32. 76
33. 82
34. 82
35. 77

Trial 3
0. 65
1. 83
2. 78
3. 70
4. 79
5. 77
6. 73
7. 75
8. 77
9. 79
10. 73
11. 74
12. 79
13. 80
14. 79
15. 80
16. 79
17. 78
18. 85
19. 74
20. 78
21. 75
22. 78
23. 83
24. 77
25. 77
26. 78
27. 77
28. 86
29. 84
30. 

# Exploit with RRP

In [4]:
# Load dataset and initialize
dataset = 'data/optimize/misc/auto93.csv'
d = DATA().adds(csv(dataset))

print(dataset)
print(len(d.rows))

def exploit(B, R):
    return B

def explore(B, R):
    return (B + R) / (abs(B - R) + 10 ** -30)

the.Last = 40
first_iterations = []
last_iterations = []

the.branch = True
random.seed(10)

# Run trials
for trial in range(100):
    print(f'\nTrial {trial}')
    random.shuffle(d.rows)
    result, iteration_results = activeLearning(d, lambda B, R: exploit(B, R))

    if iteration_results:  # Store the first and last iteration results
        first_iterations.append(iteration_results[0])
        last_iterations.append(iteration_results[-1])

# Calculate and print the average values
average_first = sum(first_iterations) / len(first_iterations)
average_last = sum(last_iterations) / len(last_iterations)

print(f'\nAverage value of the first iteration across 100 trials: {average_first}')
print(f'Average value of the last iteration across 100 trials: {average_last}')

data/optimize/misc/auto93.csv
398

Trial 0
0. 78
1. 85
2. 76
3. 77
4. 71
5. 69
6. 76
7. 84
8. 82
9. 83
10. 82
11. 78
12. 75
13. 79
14. 83
15. 84
16. 74
17. 86
18. 85
19. 73
20. 75
21. 85
22. 84
23. 69
24. 82
25. 83
26. 72
27. 73
28. 83
29. 74
30. 76
31. 83
32. 78
33. 87
34. 77
35. 71

Trial 1
0. 73
1. 73
2. 65
3. 78
4. 70
5. 75
6. 71
7. 68
8. 70
9. 71
10. 77
11. 74
12. 75
13. 69
14. 72
15. 72
16. 79
17. 67
18. 74
19. 57
20. 71
21. 76
22. 69
23. 71
24. 70
25. 75
26. 65
27. 67
28. 67
29. 57
30. 75
31. 70
32. 76
33. 64
34. 68
35. 72

Trial 2
0. 65
1. 82
2. 80
3. 79
4. 81
5. 78
6. 79
7. 80
8. 73
9. 78
10. 81
11. 83
12. 80
13. 83
14. 75
15. 79
16. 76
17. 82
18. 85
19. 80
20. 81
21. 81
22. 83
23. 77
24. 87
25. 80
26. 74
27. 78
28. 79
29. 82
30. 74
31. 70
32. 76
33. 77
34. 82
35. 83

Trial 3
0. 69
1. 83
2. 78
3. 72
4. 76
5. 82
6. 81
7. 77
8. 80
9. 81
10. 76
11. 81
12. 78
13. 74
14. 74
15. 78
16. 76
17. 75
18. 72
19. 67
20. 78
21. 78
22. 76
23. 85
24. 80
25. 82
26. 80
27. 85
28. 84
29. 83
30. 

# Explore with Random Initialization

In [13]:
# Load dataset and initialize
dataset = 'data/optimize/misc/auto93.csv'
d = DATA().adds(csv(dataset))

print(dataset)
print(len(d.rows))

def exploit(B, R):
    return B

def explore(B, R):
    return (B + R) / (abs(B - R) + 10 ** -30)

the.Last = 40
first_iterations = []
last_iterations = []

the.branch = False
random.seed(10)

# Run trials
for trial in range(100):
    print(f'\nTrial {trial}')
    random.shuffle(d.rows)
    result, iteration_results = activeLearning(d, lambda B, R: explore(B, R))

    if iteration_results:  # Store the first and last iteration results
        first_iterations.append(iteration_results[0])
        last_iterations.append(iteration_results[-1])

# Calculate and print the average values
average_first = sum(first_iterations) / len(first_iterations)
average_last = sum(last_iterations) / len(last_iterations)

print(f'\nAverage value of the first iteration across 100 trials: {average_first}')
print(f'Average value of the last iteration across 100 trials: {average_last}')

data/optimize/misc/auto93.csv
398

Trial 0
0. 75
1. 73
2. 77
3. 70
4. 81
5. 76
6. 74
7. 76
8. 80
9. 65
10. 82
11. 76
12. 78
13. 79
14. 78
15. 82
16. 79
17. 65
18. 72
19. 76
20. 79
21. 76
22. 74
23. 76
24. 69
25. 80
26. 75
27. 75
28. 72
29. 81
30. 80
31. 70
32. 72
33. 71
34. 75
35. 80

Trial 1
0. 81
1. 76
2. 78
3. 76
4. 85
5. 84
6. 78
7. 79
8. 75
9. 72
10. 78
11. 80
12. 87
13. 79
14. 74
15. 74
16. 79
17. 83
18. 73
19. 75
20. 77
21. 75
22. 81
23. 78
24. 76
25. 68
26. 81
27. 83
28. 77
29. 81
30. 75
31. 82
32. 83
33. 72
34. 82
35. 69

Trial 2
0. 76
1. 67
2. 66
3. 76
4. 73
5. 76
6. 73
7. 78
8. 73
9. 65
10. 73
11. 72
12. 73
13. 74
14. 72
15. 77
16. 65
17. 71
18. 68
19. 64
20. 69
21. 62
22. 69
23. 68
24. 64
25. 61
26. 65
27. 55
28. 63
29. 73
30. 66
31. 76
32. 68
33. 69
34. 65
35. 73

Trial 3
0. 65
1. 79
2. 79
3. 68
4. 77
5. 77
6. 67
7. 72
8. 66
9. 58
10. 66
11. 69
12. 76
13. 70
14. 62
15. 71
16. 67
17. 71
18. 65
19. 70
20. 59
21. 58
22. 57
23. 65
24. 61
25. 59
26. 69
27. 55
28. 68
29. 64
30. 

In [15]:
# Load dataset and initialize
dataset = 'data/optimize/misc/auto93.csv'
d = DATA().adds(csv(dataset))

print(dataset)
print(len(d.rows))

def exploit(B, R):
    return B

def explore(B, R):
    return (B + R) / (abs(B - R) + 10 ** -30)

the.Last = 40
first_iterations = []
last_iterations = []

the.branch = True
random.seed(10)

# Run trials
for trial in range(100):
    print(f'\nTrial {trial}')
    random.shuffle(d.rows)
    result, iteration_results = activeLearning(d, lambda B, R: explore(B, R))

    if iteration_results:  # Store the first and last iteration results
        first_iterations.append(iteration_results[0])
        last_iterations.append(iteration_results[-1])

# Calculate and print the average values
average_first = sum(first_iterations) / len(first_iterations)
average_last = sum(last_iterations) / len(last_iterations)

print(f'\nAverage value of the first iteration across 100 trials: {average_first}')
print(f'Average value of the last iteration across 100 trials: {average_last}')

data/optimize/misc/auto93.csv
398

Trial 0
0. 78
1. 78
2. 76
3. 82
4. 75
5. 71
6. 74
7. 83
8. 66
9. 78
10. 73
11. 71
12. 68
13. 73
14. 80
15. 82
16. 74
17. 78
18. 74
19. 72
20. 68
21. 76
22. 80
23. 68
24. 77
25. 70
26. 70
27. 67
28. 75
29. 72
30. 74
31. 82
32. 74
33. 82
34. 73
35. 71

Trial 1
0. 73
1. 73
2. 65
3. 76
4. 69
5. 72
6. 74
7. 67
8. 73
9. 77
10. 81
11. 72
12. 74
13. 76
14. 86
15. 68
16. 83
17. 66
18. 75
19. 68
20. 77
21. 79
22. 79
23. 77
24. 73
25. 72
26. 70
27. 72
28. 61
29. 60
30. 69
31. 75
32. 74
33. 68
34. 63
35. 77

Trial 2
0. 65
1. 82
2. 80
3. 79
4. 78
5. 72
6. 72
7. 75
8. 74
9. 72
10. 76
11. 75
12. 73
13. 75
14. 73
15. 75
16. 64
17. 80
18. 77
19. 73
20. 73
21. 80
22. 81
23. 68
24. 77
25. 82
26. 81
27. 78
28. 76
29. 77
30. 63
31. 71
32. 68
33. 66
34. 72
35. 80

Trial 3
0. 69
1. 83
2. 78
3. 72
4. 76
5. 82
6. 81
7. 77
8. 78
9. 82
10. 79
11. 80
12. 77
13. 70
14. 79
15. 74
16. 72
17. 71
18. 70
19. 66
20. 75
21. 82
22. 73
23. 76
24. 77
25. 80
26. 72
27. 72
28. 77
29. 78
30. 

# Results Summary

Exploit with random: 73.93 -> 79.16
Exploit with rrp:    51.54 -> 92.18
Explore with random: 73.93 -> 84.61
Explore with rrp:    51.54 -> 90.07

UCB with random:     80.26 -> 79.50
UCB with rrp:        80.25 -> 78.34